In [11]:
import degirum as dg, degirum_tools
#import os
from pathlib import Path
import gen_test_image_util as imgutil
#import importlib
import os
import time
import cv2
#import threading
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
#importlib.reload(imgutil)

In [12]:
#hw_location = "@cloud"
hw_location = "@local"
#model_zoo_url = "https://cs.degirum.com/degirum/public"
model_zoo_url = "https://cs.degirum.com/degirum/orca_models"
#model_zoo_url = "https://cs.degirum.com/degirum/ultralytics_v6"
device_family = "ORCA1"  # models of which device family to use
exclude_preprocessing = True
use_jpeg = True

image_dir = Path('/home/gotom/data_dir/split_pic')

num_of_pics = 200
#max_count = 3000

In [13]:
def partial_transaction(model_instance, image_chunk_pathL, start_index, max_count):
    
    count = 0
    #print(f'start_index:{start_index}')
    try:
        for res in model_instance.predict_batch(imgutil.pics_transaction_gen(model_instance, image_chunk_pathL[start_index:start_index + max_count], exclude_preprocessing, max_count)):
            count += 1
            #pass
    except Exception as e:
        print(e)
    
    #print(f'count:{count}')
    return count

In [14]:
image_size = [224, 224, 200]

# max_count = 200
# start_indexL = [0]
# instance_count = 1
        
max_count = 100
start_indexL = [0, 100]
instance_count = 2
        
""" max_count = 50
start_indexL = [0, 50, 100, 150]
instance_count = 4 """


mobile_net_v1_durationD = {}
mobile_net_v2_durationD = {}

batch_size = 16


In [15]:
imgutil.remove_image_chunk(image_dir, 3000)
image_chunkL = imgutil.create_test_image_chunk('maami_1.jpg', image_size)
image_chunk_pathL = imgutil.save_image_chunk(image_dir, image_chunkL)
    
zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token()) # connect to AI inference engine
#target_modelsL = ['mobilenet_v1_imagenet--224x224_quant_n2x_orca1_1', 'mobilenet_v2_imagenet--224x224_quant_n2x_orca1_1']
target_modelsL = ['mobilenet_v1_imagenet--224x224_quant_n2x_orca_1', 'mobilenet_v2_imagenet--224x224_quant_n2x_orca_1']
    
exclude_preprocessing = True

durationL = []
    
for mi, model_name in enumerate(target_modelsL):   
        
    image = cv2.imread(image_chunk_pathL[0], cv2.IMREAD_COLOR)
    h,w = image.shape[:2]
    image_size[0] = h
    image_size[1] = w

    model_instanceL = [imgutil.create_model_instances(model_name, zoo, batch_size, use_jpeg) for i in range(instance_count)]
        
    for i in range(instance_count):
        model_instanceL[i](image_chunk_pathL[start_indexL[i]]) #dummy
        
    for i in range(instance_count):
        model_instanceL[i].reset_time_stats()
        
    count = 0
    countL = []
    resultL = []
        
    t = degirum_tools.Timer()
          
    with ThreadPoolExecutor(max_workers = instance_count) as exctor:
        countL = [exctor.submit(partial_transaction, model_instanceL[i], image_chunk_pathL, start_indexL[i], max_count) for i in range(instance_count)] 
    
    #print(f'countL:{countL}')
    resultL = [countL[i].result() for i in range(instance_count)]   
    print(f'resultL = {resultL}')              
    elapsed_time = round(1e3*t(),1)  
    durationL.append(elapsed_time)
        
    print(f'sum(resultL):{sum(resultL)}')
    if not(sum(resultL) == num_of_pics): 
        exit()
    
    height_width = str(h) + 'x' + str(w)
    
    if mi == 0:
        mobile_net_v1_durationD[height_width] = elapsed_time
    else:
        mobile_net_v2_durationD[height_width] = elapsed_time
            
print(f'mobile_net_v1_durationD: {mobile_net_v1_durationD}')
print(f'mobile_net_v2_durationD: {mobile_net_v2_durationD}')


resultL = [100, 100]
sum(resultL):200
resultL = [100, 100]
sum(resultL):200
mobile_net_v1_durationD: {'224x224': 772.6}
mobile_net_v2_durationD: {'224x224': 965.6}
